<a href="https://colab.research.google.com/github/dharmikmehta98/IMDB_Review_Classification_Using_BERT/blob/main/NLP_with_Bert_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP with Bert for Sentiment Analysis

### Importing the libraries

In [2]:
!pip3 install ktrain

     |████████████████████████████████| 25.3 MB 91 kB/s 
     |████████████████████████████████| 6.8 MB 56.8 MB/s 
     |████████████████████████████████| 981 kB 44.2 MB/s 
     |████████████████████████████████| 263 kB 56.0 MB/s 
     |████████████████████████████████| 1.9 MB 51.8 MB/s 
     |████████████████████████████████| 1.2 MB 50.5 MB/s 
     |████████████████████████████████| 468 kB 52.6 MB/s 
     |████████████████████████████████| 895 kB 51.8 MB/s 
     |████████████████████████████████| 3.3 MB 61.2 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.1-py3-none-any.whl size=25282876 sha256=bc34d0a5c2627fa9469797155bf291dcb7e5f45d6a7d486891944b5ff26dfa0a
  Stored in directory: /root/.cache/pip/wheels/dd/88/9f/06a5d95fa827fda1af47f5c9eca78c6233928f0455171d3711
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=89525d10bba7fe3aafcf561096dc340abd39967f905ac991d930c151ff12f817
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

In [2]:
import os.path
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

## Part 1: Data Preprocessing

### Loading the IMDB dataset

In [3]:
dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
                                  origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
                                  extract=True)
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')

84131840/84125825 [==============================] - 1s 0us/step


In [4]:
print(os.path.dirname(dataset))
print(IMDB_DATADIR)

/root/.keras/datasets
/root/.keras/datasets/aclImdb


### Creating the training and test sets

In [5]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(datadir=IMDB_DATADIR,
                                                                       classes=['pos','neg'],
                                                                       maxlen=500,
                                                                       train_test_names=['train','test'],
                                                                       preprocess_mode='bert')

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## Part 2: Building the BERT model

In [10]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


## Part 3: Training the BERT model

In [12]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [1]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)

NameError: ignored

# Part 4 : Save Trained Model

In [ ]:
#Ref : https://medium.com/analytics-vidhya/how-to-deploy-your-neural-network-model-using-ktrain-ae255b134c77

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('imdb_sentiment_model')
print('MODEL SAVED')

# Part 5:Load Trained Model

In [ ]:
import pickle
from tensorflow.keras.models import load_model
# loading preprocess and model file
features = pickle.load(open('imdb_sentiment_model/tf_model.preproc',
                            'rb'))
new_model = load_model('imdb_sentiment_model/tf_model.h5')
labels = ['pos', 'neg']

# Part 6: Predict the result

In [ ]:
text = 'hey i am spam'
preproc_text = features.preprocess([text])

In [ ]:
result = new_model.predict(preproc_text)
# OUTPUT =>array([[9.9999797e-01, 2.0015173e-06]], dtype=float32)

In [ ]:
label = labels[result[0].argmax(axis=0)]
score = ('{:.2f}'.format(round(np.max(result[0]), 2)*100))
print('LABEL :', label, 'SCORE :', score)
# OUTPUT => LABEL : ham SCORE : 100.00